# Phase 3: Llama-3-8B QLoRA 파인튜닝

Phase 2에서 생성한 `training_dataset.jsonl`을 사용하여 Unsloth 기반 QLoRA 파인튜닝을 수행하고, 16bit 병합 모델을 Google Drive에 저장합니다.


## ⚙️ 사전 조건
- `phase2_instruction_dataset.ipynb` 실행으로 `board_crawling/outputs/training_dataset.jsonl` 존재
- Colab T4 이상 GPU 할당
- Google Drive에 프로젝트 폴더 업로드 완료
- 런타임 초기화 시 아래 설치 셀 재실행


---
## 0. 필수 패키지 설치 (Unsloth + TRL + xformers)


In [4]:
%pip install -q "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
%pip install -q --no-deps "trl<0.9.0" "peft<0.10.0" "accelerate<1.0.0" "bitsandbytes<0.43.0"

print("✅ Unsloth & 관련 패키지 설치 완료")

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
✅ Unsloth & 관련 패키지 설치 완료


In [ ]:
# Google Drive 마운트
from google.colab import drive

# Google Drive 마운트 (이미 마운트되어 있다면 건너뛰어도 됩니다)
drive.mount('/content/drive', force_remount=True)

print("✅ Google Drive 마운트 완료")


✅ 로컬 환경


---
## 1. 경로 및 데이터 로드


In [ ]:
from pathlib import Path
import json
from datasets import Dataset

# Colab 환경 - Google Drive 경로 설정
PROJECT_ROOT = Path("/content/drive/MyDrive/board_crawling")
OUTPUT_DIR = PROJECT_ROOT / "outputs"
MODEL_DIR = PROJECT_ROOT / "models"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
MODEL_DIR.mkdir(parents=True, exist_ok=True)

TRAIN_JSONL = OUTPUT_DIR / "training_dataset.jsonl"
if not TRAIN_JSONL.exists():
    raise FileNotFoundError(f"{TRAIN_JSONL}가 없습니다. Phase 2를 먼저 실행하세요.")

training_data = []
with open(TRAIN_JSONL, "r", encoding="utf-8") as f:
    for line in f:
        if line.strip():
            training_data.append(json.loads(line))

print(f"✅ 데이터셋 로드 완료: {len(training_data)}개 레코드")

⚠️ 노트북 파일을 찾을 수 없습니다


In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
        texts.append(text)
    return {"text": texts}

dataset = Dataset.from_list(training_data)
dataset = dataset.map(formatting_prompts_func, batched=True)
print("✅ 데이터셋 템플릿 적용 완료")

📁 현재 작업 디렉토리: /content
⚠️ Windows 경로 접근 불가: D:\0.Sogang\6\웹데이터수집과 텍스트분석\팀플 - 서담 인기글 분석 및 모델링\board_crawling\outputs\training_dataset.jsonl
🔍 training_dataset.jsonl 파일 검색 중...
⚠️ training_dataset.jsonl을 찾지 못해 노트북 파일로 검색...
⚠️ 기본 경로 사용: board_crawling


FileNotFoundError: training_dataset.jsonl 파일을 찾을 수 없습니다.
현재 작업 디렉토리: /content
프로젝트 루트: /content/board_crawling
예상 경로: /content/board_crawling/outputs/training_dataset.jsonl

💡 해결 방법:
1. 파일이 실제로 존재하는지 확인: board_crawling/outputs/training_dataset.jsonl
2. VSCode에서 워크스페이스를 노트북이 있는 폴더로 열기
3. 로컬 Python 커널 사용 고려

In [ ]:
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input_text}

### Response:
{output}"""
        texts.append(text)
    return {"text": texts}

dataset = Dataset.from_list(training_data)
dataset = dataset.map(formatting_prompts_func, batched=True)
print("✅ 데이터셋 템플릿 적용 완료")


---
## 2. Unsloth 모델 로드 및 QLoRA 준비


In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/llama-3-8b-bnb-4bit",
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha=32,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)

print("✅ Unsloth 모델 로드 완료")


---
## 3. SFTTrainer 설정 및 학습


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir=str(OUTPUT_DIR / "finetune_logs"),
        save_strategy="epoch",
        save_total_limit=3,
    ),
)

print("✅ Trainer 설정 완료. 학습을 시작합니다...")
trainer.train()
print("✅ 학습 완료")


---
## 4. 모델 저장 (LoRA + 16bit 병합)


In [ ]:
OUTPUT_MODEL_DIR = MODEL_DIR / "llama3_popular_post_lora"
OUTPUT_MODEL_DIR.mkdir(parents=True, exist_ok=True)

# LoRA 어댑터 저장
model.save_pretrained(str(OUTPUT_MODEL_DIR))
tokenizer.save_pretrained(str(OUTPUT_MODEL_DIR))
print(f"✅ LoRA 어댑터 저장 완료: {OUTPUT_MODEL_DIR}")

# 16bit 병합 모델 저장
MERGED_DIR = OUTPUT_MODEL_DIR / "merged_16bit"
FastLanguageModel.for_inference(model)
model.save_pretrained_merged(
    MERGED_DIR,
    tokenizer,
    save_method="merged_16bit",
)
print(f"✅ 16bit 병합 모델 저장 완료: {MERGED_DIR}")


> 💡 **Tip**: 필요 시 `MERGED_DIR`를 `.zip`으로 압축하여 Google Drive/Hub 등에 업로드하면 Colab 세션이 종료되어도 보관할 수 있습니다.


---
## 5. 다음 단계
- `phase4_generation.ipynb`에서 위 모델을 로드하여 추론/인기글 생성을 수행합니다.
- 추가로 Hugging Face Hub에 업로드하려면 `model.push_to_hub()`를 활용하세요.
